## Advance Python Project Assignment:

### 1. Build a Flask app that scrapes data from multiple websites and displays it on your site.You can try to scrap websites like youtube , amazon and show data on output pages and deploy it on cloud platform .

In [ ]:
from flask import Flask, render_template
import requests
from bs4 import BeautifulSoup

app = Flask(__name__)

def scrape_wikipedia():
    url = 'https://en.wikipedia.org/wiki/Main_Page'
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # Scraping Wikipedia's featured article title and content as an example
        title = soup.find('h2', id='mp-tfa-h2').text.strip()
        content = soup.find('div', id='mp-tfa').find('p').text.strip()
        return {'title': title, 'content': content}
    else:
        return None

@app.route('/')
def index():
    data = scrape_wikipedia()
    return render_template('index.html', data=data)

if __name__ == '__main__':
    app.run(debug=True)


Create an index.html file within a templates folder to display the scraped data:

### 2. Create a Flask app that consumes data from external APIs and displays it to users. Try to find an public API which will give you a data and based on that call it and deploy it on cloud platform

In [ ]:
from flask import Flask, render_template
import requests

app = Flask(__name__)

def get_random_joke():
    url = 'https://api.chucknorris.io/jokes/random'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get('value')
    else:
        return None

@app.route('/')
def index():
    joke = get_random_joke()
    return render_template('index.html', joke=joke)

if __name__ == '__main__':
    app.run(debug=True)


Create an index.html file within a templates folder to display the fetched Chuck Norris joke:

### 3. Implement OAuth2 authentication to allow users to log in using their Google or Facebook accounts.

facebook account

In [ ]:
from flask import Flask, redirect, url_for, session
from flask_oauthlib.client import OAuth

app = Flask(__name__)
app.secret_key = 'your_secret_key_here'

# Configuration for Facebook OAuth2
app.config['FACEBOOK_ID'] = 'your_facebook_app_id'
app.config['FACEBOOK_SECRET'] = 'your_facebook_app_secret'
app.config['FACEBOOK_REDIRECT_URI'] = 'http://localhost:5000/facebook/callback'

oauth = OAuth(app)
facebook = oauth.remote_app(
    'facebook',
    consumer_key=app.config['FACEBOOK_ID'],
    consumer_secret=app.config['FACEBOOK_SECRET'],
    request_token_params={'scope': 'email'},
    base_url='https://graph.facebook.com/',
    request_token_url=None,
    access_token_method='POST',
    access_token_url='/oauth/access_token',
    authorize_url='https://www.facebook.com/dialog/oauth',
)

@app.route('/')
def index():
    return 'Welcome! <a href="/facebook/login">Login with Facebook</a>'

@app.route('/facebook/login')
def facebook_login():
    return facebook.authorize(callback=url_for('facebook_authorized', _external=True))

@app.route('/facebook/callback')
def facebook_authorized():
    resp = facebook.authorized_response()
    if resp is None or 'access_token' not in resp:
        return 'Access denied: reason={}, error={}'.format(
            request.args['error_reason'],
            request.args['error_description']
        )
    
    session['facebook_token'] = (resp['access_token'], '')
    user_info = facebook.get('/me?fields=email')
    return 'Logged in as: {}'.format(user_info.data['email'])

@facebook.tokengetter
def get_facebook_oauth_token():
    return session.get('facebook_token')

if __name__ == '__main__':
    app.run(debug=True)


google account

In [ ]:
from flask import Flask, redirect, url_for, session
from flask_oauthlib.client import OAuth

app = Flask(__name__)
app.secret_key = 'your_secret_key_here'

# Configuration for Google OAuth2
app.config['GOOGLE_ID'] = 'your_google_client_id'
app.config['GOOGLE_SECRET'] = 'your_google_client_secret'
app.config['GOOGLE_REDIRECT_URI'] = 'http://localhost:5000/google/callback'

oauth = OAuth(app)
google = oauth.remote_app(
    'google',
    consumer_key=app.config['GOOGLE_ID'],
    consumer_secret=app.config['GOOGLE_SECRET'],
    request_token_params={
        'scope': 'email'
    },
    base_url='https://www.googleapis.com/oauth2/v1/',
    request_token_url=None,
    access_token_method='POST',
    access_token_url='https://accounts.google.com/o/oauth2/token',
    authorize_url='https://accounts.google.com/o/oauth2/auth',
)

@app.route('/')
def index():
    return 'Welcome! <a href="/google/login">Login with Google</a>'

@app.route('/google/login')
def google_login():
    return google.authorize(callback=url_for('google_authorized', _external=True))

@app.route('/google/callback')
def google_authorized():
    resp = google.authorized_response()
    if resp is None or resp.get('access_token') is None:
        return 'Access denied: reason={}, error={}'.format(
            request.args['error_reason'],
            request.args['error_description']
        )
    
    session['google_token'] = (resp['access_token'], '')
    user_info = google.get('userinfo')
    return 'Logged in as: {}'.format(user_info.data['email'])

@google.tokengetter
def get_google_oauth_token():
    return session.get('google_token')

if __name__ == '__main__':
    app.run(debug=True)


### 4. Develop a recommendation system using Flask that suggests content to users based on their preferences.

 Creating a Flask app that recommends movies based on user preferences:

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Movie data - dictionary of movies and their genres
movie_data = {
    'movie1': ['Action', 'Adventure', 'Sci-Fi'],
    'movie2': ['Drama', 'Romance'],
    'movie3': ['Comedy', 'Romance'],
    # Add more movies with their genres
}

# Dummy user preferences (for demonstration)
user_preferences = ['Action', 'Sci-Fi']

@app.route('/recommend', methods=['GET'])
def recommend_movies():
    recommended_movies = []
    for movie, genres in movie_data.items():
        # Calculate similarity score based on genres
        similarity_score = len(set(user_preferences) & set(genres)) / float(len(user_preferences))
        if similarity_score > 0.5:  # Set a threshold for recommendation
            recommended_movies.append(movie)

    return jsonify({'recommended_movies': recommended_movies})

if __name__ == '__main__':
    app.run(debug=True)
